## split csv files to qcv format files per year

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import pickle
# importing required modules
from zipfile import ZipFile

In [2]:
zip_dir = './ameri_data/'
all_zipfiles = glob.glob(zip_dir + "*.zip")

In [4]:
len(all_zipfiles)

333

In [5]:
dst_dir = './output/'

In [6]:
# Get those name from the c file "one_flux\ONEFlux\oneflux_steps\qc_auto\src\dataset.c"  const char *const var_names[]
var_names = ["CO2","H2O","ZL","FC","FC_SSITC_TEST","H","H_SSITC_TEST","LE","LE_SSITC_TEST","USTAR",
"TR","SB","SC","SLE","SH","P","SW_OUT","SW_IN","NETRAD","SW_DIF","PPFD_IN","APAR","TA","PA","T_CANOPY","T_BOLE","TS","SWC","G",
"RH","WD","WS","TAU","LW_IN","NEE","VPD","itpVPD","itpSW_IN","itpPPFD_IN","itpTA","itpTS","itpSWC","itpP","itpRH","FETCH_FILTER",
]

In [7]:
must_have = ['TA','RH', 'PA', 'WS','P', 'SW_IN', 'LW_IN']

In [8]:
# map columns to qc_auto accepted columns name
# like CO2_1_1_1 to CO2
def pickup_col(df: pd.DataFrame) ->list:
    new_column = dict()
    for item in df.columns[2:]:
        if item in var_names:
            new_column[item] = item
        else:
            sub_items = item.split('_')
            key0 = sub_items[0]
            if key0 in var_names:
                if key0 not in new_column.keys():
                    new_column[key0] = item
                else:
                    continue
            else:
                if len(sub_items) == 1:
                    continue
                key1 = key0 + '_'+ sub_items[1]  # According to var_name, there only have xx_xxx format besides XX_SSITC_TEST items
                if key1 in var_names:
                    if key1 not in new_column.keys():
                        new_column[key1] = item
                    else:
                        continue
                else:
                    continue
                    
    return new_column

In [9]:
#split rows per years
# return year list and start and end position for each year
def split_records(src_data: pd.DataFrame) -> (list, list, list):
    years = []
    start_pos = []
    end_pos = []

    start_year = ''
    rows = src_data['TIMESTAMP_END'].count()
    for idx in range(rows):
        if start_year == '':
            start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
            years.append(start_year)
            start_pos.append(idx)
        else:
            if start_year == src_data.iloc[idx]['TIMESTAMP_START'][:4]:
                _end = idx
            else:
                end_pos.append(_end)
                start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
                years.append(start_year)
                start_pos.append(idx)

    end_pos.append(_end)
    
    return years, start_pos, end_pos

In [10]:
def create_vpd(row):
    ta = row['TA']
    rh = row['RH']
    if ta == -9999 or rh == -9999:
        vpd =  -9999
    else:
        es = 0.6108 * np.exp((17.27*ta)/(ta+237.3)) * 1000
        ea = es * rh / 100.0
        vpd = (es - ea) /100.0

    return round(vpd,4)

In [11]:
def create_qcv_files(df: pd.DataFrame,site_info: pd.DataFrame, dst_name:str, site_name: str, record_interval: str):
    
    new_columns = pickup_col(df)
    
   
    header = ['TIMESTAMP_START','TIMESTAMP_END']
    # revert key value position
    selected_cols = dict()
    for key,value in new_columns.items():
        selected_cols[value] = key
    
    header += list(selected_cols.keys())
    
    years, start_pos, end_pos = split_records(df)
    
    #site_name = 'US-Ne2'
    for year, start, end in zip(years, start_pos, end_pos):
        if int(year) > 2021:
            continue
        
        # file name format is like US-ARc_qcv_2005.csv
        file_name = dst_name + site_name + '_qcv_'+ year + '.csv'
        _data = df[header].iloc[start: end+1].rename(columns=selected_cols)
        _data['VPD'] = _data.apply (lambda row: create_vpd(row) , axis=1)
        _data.to_csv(file_name,index=False)
        
        file = open(file_name,'r')
        text = file.read()
        file.close()
        file = open(file_name,'w')
        file.write('site,{}\n'.format(site_name))
        file.write('year,{}\n'.format(year))
        file.write('lat,{}\n'.format(site_info['Latitude'].values[0]))
        file.write('lon,{}\n'.format(site_info['Longitude'].values[0]))
        file.write('timezone,{}\n'.format(site_info['UTC'].values[0]))
        file.write('htower,{},{}\n'.format(_data.iloc[0]['TIMESTAMP_END'], 5))
        file.write('timeres,{}\n'.format(record_interval))
        file.write('sc_negl,1\n')
        file.write('notes,202305041205 qc visual comparison SY\n')
        file.write(text)
        file.close()
        
        print('...... Create file {} successful.'.format(file_name))
    
    return

In [12]:
pd_site = pd.read_csv('AmeriFlux_siteinfo.csv')
pd_site

,site_id,Latitude,Longitude,Elev,UTC
0,AR-TF1,-54.9733,-66.7335,40.0,-3.0
1,AR-TF2,-54.8269,-68.4549,60.0,-3.0
2,BR-CST,-7.9682,-38.3842,468.0,-3.0
3,BR-Npw,-16.4980,-56.4120,120.0,-4.0
4,CA-ARB,52.6950,-83.9452,90.0,-5.0
...,...,...,...,...,...
328,US-xUK,39.0404,-95.1921,335.0,-6.0
329,US-xUN,46.2339,-89.5373,518.0,-6.0
330,US-xWD,47.1282,-99.2414,579.0,-6.0
331,US-xWR,45.8205,-121.9519,407.0,-8.0


In [13]:
for era_site_list in os.walk('./output/'):
    break

era_site_list[1]

['US-xWR',
 'US-Syv',
 'US-CC2',
 'US-MC1',
 'US-Kon',
 'US-Dk3',
 'US-Bo2',
 'US-xJE',
 'US-CRT',
 'US-TrS',
 'US-xSJ',
 'US-Wi1',
 'US-DFK',
 'US-SRG',
 'US-Wlr',
 'US-DPW',
 'US-PFf',
 'CA-TP3',
 'US-MOz',
 'US-UiA',
 'US-Wi8',
 'US-GLE',
 'US-PSH',
 'US-PFh',
 'US-MtB',
 'US-CF2',
 'US-xST',
 'US-MVF',
 'US-xSL',
 'US-Jo1',
 'US-Ho3',
 'US-Mi3',
 'US-MMS',
 'CA-TP2',
 'US-Lin',
 'US-Wi3',
 'US-xSR',
 'US-Ne1',
 'US-Dia',
 'US-PHM',
 'US-PFa',
 'US-Ha2',
 'US-xDL',
 'US-RGo',
 'US-Pnp',
 'US-JRn',
 'US-Ho2',
 'US-Bi1',
 'CA-MA1',
 'US-Fuf',
 'US-MVW',
 'US-KFS',
 'US-Oho',
 'US-Mpj',
 'US-SP2',
 'US-Blo',
 'CA-DB2',
 'US-AR1',
 'US-Akn',
 'US-xJR',
 'US-PFd',
 'US-NR1',
 'US-CS5',
 'US-Vcp',
 'US-Rms',
 'US-Rwf',
 'US-xSP',
 'CA-MA3',
 'US-xUK',
 'CA-ARB',
 'US-TrB',
 'US-xUN',
 'US-xRM',
 'US-PFc',
 'MX-Aog',
 'US-Mj1',
 'US-CF4',
 'US-xBL',
 'US-DS3',
 'US-Vcm',
 'US-NC4',
 'US-Vcs',
 'US-Ro5',
 'US-Wi9',
 'US-Wrc',
 'US-CS3',
 'US-RGA',
 'US-xWD',
 'US-Tw3',
 'US-xSC',
 'US-ASM',

In [14]:
len(era_site_list[1])

284

In [15]:
processed_list = []
bad_list = [] # some site data missing necessary columns
for zip_file in all_zipfiles:
    #if zip_file in processed_list:
    #    continue
        
    #if zip_file in bad_list:
    #    continue
    
    
    file_name = os.path.basename(zip_file)

    
    file_name = file_name.split('.')[0]
    site_name = file_name.split('_')[1]
    
    #don't create qcv files if no era file existing
    if site_name not in era_site_list[1]:
        continue
    
    site_info = pd_site[pd_site['site_id'] == site_name]
    
    dst_name = dst_dir + site_name + '/01_qc_visual/qcv_files/'
    if not os.path.exists(dst_name):
        os.makedirs(dst_name)
        print("Create new dir ", dst_name)


    with ZipFile(zip_file, 'r') as zip_folder:
        file_list = zip_folder.namelist()
        if len(file_list) > 1:
            for _file in file_list:
                name_split = _file.split('.')

                if name_split[1] == 'csv':
                    df = pd.read_csv(zip_folder.open(_file),dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str},skiprows=2)
                    if df.iloc[0]['TIMESTAMP_END'][-4:] == '0030':
                        record_interval = 'halfhourly'
                    else:
                        record_interval = 'hourly'
                        
                    print('Process zip file {}........'.format(zip_file))
                    
                    # check data 
                    new_columns = pickup_col(df)
                    
                    # if necessary column missing, add it with value -9999
                    missing_list = []
                    bad_dict = dict()
                    for x in must_have:
                        if x not in new_columns:
                            df[x] = -9999.0
                            missing_list.append(x)
                            
                    if len(missing_list) > 0:
                        bad_dict[zip_file] = missing_list
                        bad_list.append(bad_dict)
                        

                    # check must have columns
                    #flag = 0
                    #if(all(x in new_columns for x in must_have)):
                    #    flag = 1
                    #if flag == 0:
                    #    bad_list.append(zip_file)
                    #    break
                    
                    #those are for create files
                    create_qcv_files(df,site_info, dst_name, site_name, record_interval)
                    
    processed_list.append(zip_file)

Create new dir  ./output/US-xGR/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-xGR_BASE-BADM_6-5.zip........
...... Create file ./output/US-xGR/01_qc_visual/qcv_files/US-xGR_qcv_2017.csv successful.
...... Create file ./output/US-xGR/01_qc_visual/qcv_files/US-xGR_qcv_2018.csv successful.
...... Create file ./output/US-xGR/01_qc_visual/qcv_files/US-xGR_qcv_2019.csv successful.
...... Create file ./output/US-xGR/01_qc_visual/qcv_files/US-xGR_qcv_2020.csv successful.
...... Create file ./output/US-xGR/01_qc_visual/qcv_files/US-xGR_qcv_2021.csv successful.
Create new dir  ./output/US-xAE/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-xAE_BASE-BADM_5-5.zip........
...... Create file ./output/US-xAE/01_qc_visual/qcv_files/US-xAE_qcv_2017.csv successful.
...... Create file ./output/US-xAE/01_qc_visual/qcv_files/US-xAE_qcv_2018.csv successful.
...... Create file ./output/US-xAE/01_qc_visual/qcv_files/US-xAE_qcv_2019.csv successful.
...... Create file ./output/US-xAE

/home/minan/anaconda3/envs/oneflux/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Process zip file ./ameri_data/AMF_US-KM1_BASE-BADM_4-5.zip........
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2009.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2010.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2011.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2012.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2013.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2014.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2015.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2016.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2017.csv successful.
...... Create file ./output/US-KM1/01_qc_visual/qcv_files/US-KM1_qcv_2018.csv successful.
...... Create file ./output/US-KM

...... Create file ./output/US-Dix/01_qc_visual/qcv_files/US-Dix_qcv_2008.csv successful.
Create new dir  ./output/US-CPk/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-CPk_BASE-BADM_2-1.zip........
...... Create file ./output/US-CPk/01_qc_visual/qcv_files/US-CPk_qcv_2009.csv successful.
...... Create file ./output/US-CPk/01_qc_visual/qcv_files/US-CPk_qcv_2010.csv successful.
...... Create file ./output/US-CPk/01_qc_visual/qcv_files/US-CPk_qcv_2011.csv successful.
...... Create file ./output/US-CPk/01_qc_visual/qcv_files/US-CPk_qcv_2012.csv successful.
...... Create file ./output/US-CPk/01_qc_visual/qcv_files/US-CPk_qcv_2013.csv successful.
Create new dir  ./output/US-HB3/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-HB3_BASE-BADM_1-5.zip........
...... Create file ./output/US-HB3/01_qc_visual/qcv_files/US-HB3_qcv_2019.csv successful.
Create new dir  ./output/US-Fwf/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-Fwf_BASE-BADM_8-5.zip........
.

...... Create file ./output/US-Vcs/01_qc_visual/qcv_files/US-Vcs_qcv_2020.csv successful.
...... Create file ./output/US-Vcs/01_qc_visual/qcv_files/US-Vcs_qcv_2021.csv successful.
Create new dir  ./output/US-MOz/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-MOz_BASE-BADM_11-5.zip........
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2004.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2005.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2006.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2007.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2008.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2009.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_qcv_2010.csv successful.
...... Create file ./output/US-MOz/01_qc_visual/qcv_files/US-MOz_q

...

...... Create file ./output/US-xAB/01_qc_visual/qcv_files/US-xAB_qcv_2017.csv successful.
...... Create file ./output/US-xAB/01_qc_visual/qcv_files/US-xAB_qcv_2018.csv successful.
...... Create file ./output/US-xAB/01_qc_visual/qcv_files/US-xAB_qcv_2019.csv successful.
...... Create file ./output/US-xAB/01_qc_visual/qcv_files/US-xAB_qcv_2020.csv successful.
...... Create file ./output/US-xAB/01_qc_visual/qcv_files/US-xAB_qcv_2021.csv successful.
Create new dir  ./output/US-Ne2/01_qc_visual/qcv_files/
Process zip file ./ameri_data/AMF_US-Ne2_BASE-BADM_13-5.zip........
...... Create file ./output/US-Ne2/01_qc_visual/qcv_files/US-Ne2_qcv_2001.csv successful.
...... Create file ./output/US-Ne2/01_qc_visual/qcv_files/US-Ne2_qcv_2002.csv successful.
...... Create file ./output/US-Ne2/01_qc_visual/qcv_files/US-Ne2_qcv_2003.csv successful.
...... Create file ./output/US-Ne2/01_qc_visual/qcv_files/US-Ne2_qcv_2004.csv successful.
...... Create file ./output/US-Ne2/01_qc_visual/qcv_files/US-Ne2_q

Process zip file ./ameri_data/AMF_CA-Ca3_BASE-BADM_4-5.zip........
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2001.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2002.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2003.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2004.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2005.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2006.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2007.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2008.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2009.csv successful.
...... Create file ./output/CA-Ca3/01_qc_visual/qcv_files/CA-Ca3_qcv_2010.csv successful.
...... Create file ./output/CA-Ca

...... Create file ./output/US-xDL/01_qc_visual/qcv_files/US-xDL_qcv_2018.csv successful.
...... Create file ./output/US-xDL/01_qc_visual/qcv_files/US-xDL_qcv_2019.csv successful.
...... Create file ./output/US-xDL/01_qc_visual/qcv_files/US-xDL_qcv_2020.csv successful.
...... Create file ./output/US-xDL/01_qc_visual/qcv_files/US-xDL_qcv_2021.csv successful.
Create new dir  ./output/US-KL1/01_qc_visual/qcv_files/


/home/minan/anaconda3/envs/oneflux/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (21,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Process zip file ./ameri_data/AMF_US-KL1_BASE-BADM_3-5.zip........
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2009.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2010.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2011.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2012.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2013.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2014.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2015.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2016.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2017.csv successful.
...... Create file ./output/US-KL1/01_qc_visual/qcv_files/US-KL1_qcv_2018.csv successful.
...... Create file ./output/US-KL

In [21]:
bad_list

[{'./ameri_data/AMF_US-PFs_BASE-BADM_1-5.zip': ['P']},
 {'./ameri_data/AMF_US-JRn_BASE-BADM_1-5.zip': ['P', 'SW_IN', 'LW_IN']},
 {'./ameri_data/AMF_US-KM1_BASE-BADM_4-5.zip': ['P', 'LW_IN']},
 {'./ameri_data/AMF_US-Tw5_BASE-BADM_3-5.zip': ['P']},
 {'./ameri_data/AMF_US-UiA_BASE-BADM_2-5.zip': ['P']},
 {'./ameri_data/AMF_US-Mi3_BASE-BADM_1-5.zip': ['RH', 'SW_IN', 'LW_IN']},
 {'./ameri_data/AMF_US-PSH_BASE-BADM_1-5.zip': ['P', 'LW_IN']},
 {'./ameri_data/AMF_US-SRS_BASE-BADM_3-5.zip': ['LW_IN']},
 {'./ameri_data/AMF_US-TrB_BASE-BADM_1-5.zip': ['TA', 'RH', 'PA', 'P']},
 {'./ameri_data/AMF_US-Cop_BASE-BADM_2-5.zip': ['LW_IN']},
 {'./ameri_data/AMF_US-Tur_BASE-BADM_1-5.zip': ['PA', 'P']},
 {'./ameri_data/AMF_US-Tw1_BASE-BADM_9-5.zip': ['P']},
 {'./ameri_data/AMF_US-MH1_BASE-BADM_1-5.zip': ['PA', 'P']},
 {'./ameri_data/AMF_US-BMM_BASE-BADM_3-5.zip': ['PA']},
 {'./ameri_data/AMF_US-CF1_BASE-BADM_3-5.zip': ['SW_IN', 'LW_IN']},
 {'./ameri_data/AMF_US-Bar_BASE-BADM_5-5.zip': ['PA', 'LW_IN']},
 {'

In [23]:
with open('bad_list.txt', 'w') as f:
    for line in bad_list:
        f.write('{}\n'.format(line))
        
print('Done!')

Done!


### Below code for easy debeuggin


#### For running OneFluxnet code, the 06_meteo last year is 2021, so 01_qcv require last year will not more than 2021
#### below code to remove records like 2022


In [ ]:
src_dir = './output/'
qcv_dir = '/01_qc_visual/qcv_files/'

In [ ]:
for sub_dir_list in os.walk(src_dir):
    break

sub_dir_list[1]

In [ ]:
for sub_dir in sub_dir_list[1]:
    site_name = sub_dir
    
    full_qcv_dir = src_dir + site_name + qcv_dir
    qcv_file_list = glob.glob(full_qcv_dir + '*.csv')
    years = []
    for qcv_file in qcv_file_list:
        qcv_name = os.path.basename(qcv_file)
        year = qcv_name.split('.')[0][-4:]
        year = int(year)
        if year > 2021:
            print("Will remove ", qcv_file)
            os.remove(qcv_file) 


#### below code for change Hourly records to half hour records

In [41]:
# AMF_US-PFa_BASE-BADM_22-5.zip
# this is hourly records, need change to half hour records.
#csv_files = glob.glob('./output/US-PFa/01_qc_visual/qcv_files/*.csv')
#csv_files = glob.glob('./output/US-MMS/01_qc_visual/qcv_files/*.csv')
#csv_files = glob.glob('./output/US-Ne1/01_qc_visual/qcv_files/*.csv')
#csv_files = glob.glob('./output/US-Ne3/01_qc_visual/qcv_files/*.csv')
#csv_files = glob.glob('./output/US-Ne2/01_qc_visual/qcv_files/*.csv')
#csv_files = glob.glob('./output/US-Ha1/01_qc_visual/qcv_files/*.csv')
csv_files = glob.glob('./output/US-Cop/01_qc_visual/qcv_files/*.csv')


In [42]:
for qcv_file in csv_files:
    
    df_qcv = pd.read_csv(qcv_file,dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str},skiprows=9)
    df_qcv = df_qcv.replace(-9999.0, np.nan)
    
    start_day = df_qcv.iloc[0]['TIMESTAMP_START']
    end_day = df_qcv.iloc[-1]['TIMESTAMP_END']
    T = pd.date_range(start_day, end_day,freq='30min')
    T0 = pd.DatetimeIndex(df_qcv['TIMESTAMP_START'])
    
    new_index = pd.RangeIndex(T.size-1)
    new_df = pd.DataFrame(np.nan, index= new_index, columns=df_qcv.columns)
    
    new_df.iloc[new_df.index[np.in1d(T[:-1],T0)]] = df_qcv
    
    for i in range(T.size-1):
        start_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i].year,T[i].month,T[i].day,T[i].hour, T[i].minute)
        end_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i+1].year,T[i+1].month,T[i+1].day,T[i+1].hour, T[i+1].minute)
        new_df.loc[i,'TIMESTAMP_START'] = start_time
        new_df.loc[i,'TIMESTAMP_END'] = end_time
        
    # The precipitation is accumulated value
    if 'P' in new_df.columns:
        new_df['P'] = new_df['P']/2
    
    new_df = new_df.interpolate()
    new_df = new_df.replace(np.nan, -9999.0)
    
    # read first 9 lines from the qcv csv file
    file1 = open(qcv_file, 'r')
    count = 0
    pre_lines = []
    for line in file1:
        count += 1
        #print("Line{}: {}".format(count, line.strip()))
        if count == 7:
            new_line = 'timeres,halfhourly'
            pre_lines.append(new_line)
        else:
            pre_lines.append(line.strip())
        if count == 9:
            break

    # Closing files
    file1.close()

    
    # update rows
    new_df.to_csv(qcv_file, index=False)

    file = open(qcv_file,'r')
    text = file.read()
    file.close()

    with open(qcv_file,'w') as f:
        f.write('\n'.join(pre_lines))
        f.write('\n')
        f.write(text)

In [31]:
pre_lines

['site,US-Ne1',
 'year,2010',
 'lat,41.1651',
 'lon,-96.4766',
 'timezone,-6.0',
 'htower,201001010100,5',
 'timeres,halfhourly',
 'sc_negl,1',
 'notes,202305041205 qc visual comparison SY']

In [27]:
qcv_file = csv_files[1]

In [29]:
# read first 9 lines from the qcv csv file
file1 = open(qcv_file, 'r')
count = 0
pre_lines = []
for line in file1:
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    if count == 7:
        new_line = 'timeres,halfhourly'
        pre_lines.append(new_line)
    else:
        pre_lines.append(line.strip())
    if count == 9:
        break

Line1: site,US-Ne1
Line2: year,2017
Line3: lat,41.1651
Line4: lon,-96.4766
Line5: timezone,-6.0
Line6: htower,201701010100,5
Line7: timeres,hourly
Line8: sc_negl,1
Line9: notes,202305041205 qc visual comparison SY


In [ ]:
df_qcv

In [ ]:
new_df[:20]

In [ ]:
if 'P' in new_df.columns:
    print('P')

In [ ]:
new_df['P']

In [ ]:
new_df = new_df.replace(np.nan, -9999.0)
new_df

In [ ]:
start_time = df_qcv.iloc[0]['TIMESTAMP_START']
end_time = df_qcv.iloc[-1]['TIMESTAMP_END']
T = pd.date_range(start_time, end_time,freq='30min')
T

In [ ]:
T.size

In [ ]:
new_index = pd.RangeIndex(T.size-1)
new_df = pd.DataFrame(np.nan, index= new_index, columns=df_qcv.columns)
new_df

In [ ]:
T0 = pd.DatetimeIndex(df_qcv['TIMESTAMP_START'])
T0

In [ ]:
new_df.iloc[new_df.index[np.in1d(T[:-1],T0)]] = df_qcv

In [ ]:
for i in range(T.size-1):
    start_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i].year,T[i].month,T[i].day,T[i].hour, T[i].minute)
    end_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i+1].year,T[i+1].month,T[i+1].day,T[i+1].hour, T[i+1].minute)
    new_df.loc[i,'TIMESTAMP_START'] = start_time
    new_df.loc[i,'TIMESTAMP_END'] = end_time
new_df

In [ ]:
#new_df[np.in1d(T[:-1],T0)] = df_qcv
new_df

In [ ]:
new_df.head()

In [ ]:
np.in1d(T0,T).sum()

In [ ]:
T0 = pd.DatetimeIndex(df['Timestamp'])

Ta = np.full(T.size,np.nan,np.float32)
Ta[np.in1d(T,T0)] = df['tair'][np.in1d(T0,T)]

In [ ]:
bad_list_files = [os.path.basename(item) for item in bad_list]
bad_list_files
pickle.dump( bad_list_files, open( "bad_list.pkl", "wb" ) )

In [ ]:
site_info['Latitude'].values[0]

In [ ]:
bad_list = pickle.load( open( "bad_list.pkl", "rb" ) )
bad_list

In [ ]:
for era_site_list in os.walk('./output/'):
    break

era_site_list[1]

In [ ]:
os.path.basename(zip_file)

In [ ]:
src_file = 'US-Ne2/AMF_US-Ne2_BASE_HR_9-5.csv'
src_data = pd.read_csv(src_file, dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str},skiprows=2)
src_data

In [ ]:
src_data.iloc[0]['TIMESTAMP_END']

In [ ]:
df = src_data

In [ ]:
new_column = dict()
for item in df.columns[2:]:
    if item in var_names:
        new_column[item] = item
    else:
        sub_items = item.split('_')
        key0 = sub_items[0]
        if key0 in var_names:
            if key0 not in new_column.keys():
                new_column[key0] = item
            else:
                continue
        else:
            key1 = key0 + '_'+ sub_items[1]  # According to var_name, there only have xx_xxx format besides XX_SSITC_TEST items
            if key1 in var_names:
                if key1 not in new_column.keys():
                    new_column[key1] = item
                else:
                    continue
            else:
                continue

In [ ]:
header = ['TIMESTAMP_START','TIMESTAMP_END']
selected_cols = dict()
for key,value in new_column.items():
    selected_cols[value] = key

In [ ]:
selected_cols

In [ ]:
header += list(selected_cols.keys())
header

In [ ]:
rows = src_data['TIMESTAMP_END'].count()
rows

In [ ]:
src_data.iloc[1]['TIMESTAMP_START'][:4]

In [ ]:
years = []
start_pos = []
end_pos = []

start_year = ''

for idx in range(rows):
    if start_year == '':
        start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
        years.append(start_year)
        start_pos.append(idx)
    else:
        if start_year == src_data.iloc[idx]['TIMESTAMP_START'][:4]:
            _end = idx
        else:
            end_pos.append(_end)
            start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
            years.append(start_year)
            start_pos.append(idx)
            
end_pos.append(_end)

In [ ]:
years, start_pos, end_pos

In [ ]:
site_name = 'US-Ne2'
for year, start, end in zip(years, start_pos, end_pos):
    # file name format is like US-ARc_qcv_2005.csv
    file_name = site_name + '_qcv_'+ year + '.csv'
    _data = src_data[header].iloc[start: end+1].rename(columns=selected_cols)
    _data.to_csv('./ameri_sites/US-Ne2/'+ file_name,float_format='%.5f', index=False)

In [ ]:
selected_cols

In [ ]:
_tmp = src_data[header].iloc[100: 200+1].rename(columns=selected_cols)
_tmp

In [ ]:
_tmp['TA'] = np.random.rand(101)
_tmp['RH'] = np.random.rand(101)

In [ ]:
def create_vpd(row):
    ta = row['TA']
    rh = row['RH']
    if ta == -9999 or rh == -9999:
        vpd =  -9999
    else:
        es = 0.6108 * np.exp((17.27*ta)/(ta+237.3)) * 1000
        ea = es * rh / 100.0
        vpd = (es - ea) /100.0

    return vpd
_tmp['VPD'] = _tmp.apply (lambda row: create_vpd(row) , axis=1)

In [ ]:
_tmp.columns